In [16]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import math  

In [17]:
iris = load_iris()
X = iris.data
y = iris.target

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [19]:
def kfold(train_data, y_data, k):
    n = train_data.shape[0]
    co = math.ceil(n/k)
    k_train = []
    for i in range(k-1):
        k_train.append((train_data[i*co:(i+1)*co], y_data[i*co:(i+1)*co]))
    k_train.append((train_data[(k-1)*co:], y_data[(k-1)*co:]))
    return k_train

In [20]:
def run_k_fold(k_fold_data):
    best = 0
    best_accuracy = -1
    k = len(k_fold_data)
    for i in range(k):
        xtest, ytest = k_fold_data[i]
        xtrain, ytrain = (np.zeros(xtest.shape),np.zeros(ytest.shape))
        for j in range(k):
            if i == j:
                continue
            else:
                xtrain = xtrain + k_fold_data[j][0]
                ytrain = ytrain + k_fold_data[j][1]
        learning_rate = 0.01
        no_iterations = 3000
        w, b = weightInitialization(xtrain.shape[1])
        coeff, _, _ = model_predict(w, b, xtrain, ytrain, learning_rate, no_iterations)
        final_pred = sigmoid_activation(np.dot(xtest, coeff["w"]) + coeff["b"])
        y_pred = predict(final_pred)
        accuracy = accuracy_score(ytest, y_pred)
        print(i,"Accuracy:", accuracy)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best = i
    return best

In [21]:
def weightInitialization(n_features):
    w = np.zeros((n_features, 1))
    b = 0
    return w, b

In [22]:
def sigmoid_activation(result):
    return 1 / (1 + np.exp(-np.clip(result, -100, 100)))

In [23]:
def model_optimize(w, b, X, Y):
    m = X.shape[0]
    
    # Prediction
    final_result = sigmoid_activation(np.dot(X, w) + b)
    
    epsilon = 1e-15  # Small constant to avoid division by zero
    final_result = np.clip(final_result, epsilon, 1 - epsilon)  # Clip predictions to prevent extreme values
    
    cost = (-1/m) * np.sum(Y * np.log(final_result) + (1 - Y) * np.log(1 - final_result))
    
    # Gradient calculation
    dw = (1/m) * np.dot(X.T, (final_result - Y))
    db = (1/m) * np.sum(final_result - Y)
    
    grads = {"dw": dw, "db": db}
    
    return grads, cost

In [24]:
def model_predict(w, b, X, Y, learning_rate, no_iterations):
    costs = []
    for i in range(no_iterations):
        grads, cost = model_optimize(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        
        # Weight update
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)
    
    coeff = {"w": w, "b": b}
    gradient = {"dw": dw, "db": db}
    
    return coeff, gradient, costs

In [25]:
def predict(final_pred):
    # return final_pred
    y_preds = []
    for pred in final_pred:
        y_pred = np.zeros((1, pred.shape[0]))
        for i in range(pred.shape[0]):
            if pred[0][i] > 0.5:
                y_pred[0][i] = 1
        y_preds.append(y_pred)
    return y_preds

In [26]:
def change(y_preds):
    for i in range(len(y_preds)):
        for j in range(len(y_preds[i][0])):
            if y_preds[i][0][j] == 0:
                y_preds[i][0][j] = -1
            elif y_preds[i][0][j] == 1:
                y_preds[i][0][j] = i
    y_pred = y_preds[0][0]
    for i in range(len(y_preds)):
        for j in range(len(y_preds[i][0])):
            if y_pred[j] == -1 and y_preds[i][0][j] !=-1:
                y_pred[j] = y_preds[i][0][j]
    return y_pred

In [37]:
learning_rate = 0.001
no_iterations = 5000

In [38]:
num_classes = len(np.unique(y_train))
weights = []
biases = []

for _ in range(num_classes):
    w, b = weightInitialization(X_train.shape[1])
    weights.append(w)
    biases.append(b)

In [39]:
coeffs = []
for i in range(num_classes):
    y_train_class = (y_train == i).astype(int)
    coeff, _, _ = model_predict(weights[i], biases[i], X_train, y_train_class, learning_rate, no_iterations)
    coeffs.append(coeff)

In [40]:
final_preds = []
for i in range(num_classes):
    final_pred = sigmoid_activation(np.dot(X_test, coeffs[i]["w"]) + coeffs[i]["b"])
    final_preds.append(final_pred)

In [42]:
print(final_preds)
y_preds = predict(final_preds)
y_pred = change(y_preds)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
y_pred,y_test

[array([[6.51780160e-05, 6.51780160e-05, 6.51780160e-05, ...,
        6.51780160e-05, 9.99966677e-01, 9.99966677e-01],
       [8.24523922e-04, 8.24523922e-04, 8.24523922e-04, ...,
        8.24523922e-04, 9.98433315e-01, 9.98433315e-01],
       [1.76642116e-05, 1.76642116e-05, 1.76642116e-05, ...,
        1.76642116e-05, 9.99995685e-01, 9.99995685e-01],
       ...,
       [5.29552325e-04, 5.29552325e-04, 5.29552325e-04, ...,
        5.29552325e-04, 9.99233349e-01, 9.99233349e-01],
       [1.22824074e-04, 1.22824074e-04, 1.22824074e-04, ...,
        1.22824074e-04, 9.99913005e-01, 9.99913005e-01],
       [3.75809664e-05, 3.75809664e-05, 3.75809664e-05, ...,
        3.75809664e-05, 9.99986374e-01, 9.99986374e-01]]), array([[9.99962328e-01, 9.99962328e-01, 6.20714204e-05, ...,
        9.99962328e-01, 6.20714204e-05, 6.20714204e-05],
       [9.98519707e-01, 9.98519707e-01, 9.13734182e-04, ...,
        9.98519707e-01, 9.13734182e-04, 9.13734182e-04],
       [9.99994620e-01, 9.99994620e-01, 1

(array([1., 1., 2., 0., 0., 1., 0., 1., 2., 2., 0., 1., 2., 2., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 2., 2., 1., 2., 1., 2., 2.]),
 array([2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 1, 1, 1, 2, 2, 2, 0,
        2, 0, 1, 2, 1, 0, 1, 2]))